<a href="https://colab.research.google.com/github/ab3131/NBAInjuryPredictor/blob/main/NBAInjuryPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Statements

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import classy_classification
import spacy
from spacy.language import Language

# Data Cleaning

In [2]:
player_stats = pd.read_csv('player_stats.csv')
injuries_original = pd.read_csv('NBA Player Injury Stats(1951 - 2023).csv')

In [3]:
player_stats.columns[0]


'Unnamed: 0'

In [4]:
injuries_original.columns[0]

'Unnamed: 0'

In [5]:
player_stats = player_stats.drop(columns = player_stats.columns[0], axis=1)

In [6]:
player_stats

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,Randy Livingston,HOU,22.0,193.04,94.800728,Louisiana State,USA,1996,2,42,...,3.9,1.5,2.4,0.3,0.042,0.071,0.169,0.487,0.248,1996-97
1,Gaylon Nickerson,WAS,28.0,190.50,86.182480,Northwestern Oklahoma,USA,1994,2,34,...,3.8,1.3,0.3,8.9,0.030,0.111,0.174,0.497,0.043,1996-97
2,George Lynch,VAN,26.0,203.20,103.418976,North Carolina,USA,1993,1,12,...,8.3,6.4,1.9,-8.2,0.106,0.185,0.175,0.512,0.125,1996-97
3,George McCloud,LAL,30.0,203.20,102.058200,Florida State,USA,1989,1,7,...,10.2,2.8,1.7,-2.7,0.027,0.111,0.206,0.527,0.125,1996-97
4,George Zidek,DEN,23.0,213.36,119.748288,UCLA,USA,1995,1,22,...,2.8,1.7,0.3,-14.1,0.102,0.169,0.195,0.500,0.064,1996-97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12839,Joel Embiid,PHI,29.0,213.36,127.005760,Kansas,Cameroon,2014,1,3,...,33.1,10.2,4.2,8.8,0.057,0.243,0.370,0.655,0.233,2022-23
12840,John Butler Jr.,POR,20.0,213.36,86.182480,Florida State,USA,Undrafted,Undrafted,Undrafted,...,2.4,0.9,0.6,-16.1,0.012,0.065,0.102,0.411,0.066,2022-23
12841,John Collins,ATL,25.0,205.74,102.511792,Wake Forest,USA,2017,1,19,...,13.1,6.5,1.2,-0.2,0.035,0.180,0.168,0.593,0.052,2022-23
12842,Jericho Sims,NYK,24.0,208.28,113.398000,Texas,USA,2021,2,58,...,3.4,4.7,0.5,-6.7,0.117,0.175,0.074,0.780,0.044,2022-23


In [7]:
injuries_original = injuries_original.drop(columns=injuries_original.columns[0], axis=1)

In [8]:
injuries_original

,Date,Team,Acquired,Relinquished,Notes
0,1951-12-25,Bullets,NaN,Don Barksdale,placed on IL
1,1952-12-26,Knicks,NaN,Max Zaslofsky,placed on IL with torn side muscle
2,1956-12-29,Knicks,NaN,Jim Baechtold,placed on inactive list
3,1959-01-16,Lakers,NaN,Elgin Baylor,player refused to play after being denied a ro...
4,1961-11-26,Lakers,NaN,Elgin Baylor,player reported for military duty
...,...,...,...,...,...
37662,2023-04-16,Clippers,Marcus Morris,NaN,activated from IL
37663,2023-04-16,Grizzlies,Dillon Brooks,NaN,activated from IL
37664,2023-04-16,Grizzlies,Ja Morant,NaN,activated from IL
37665,2023-04-16,Grizzlies,Jaren Jackson Jr.,NaN,activated from IL


Filter out injuries prior to 1996, where the player_stats dataset begins. We also don't want any rows where someone was activated, we care about the injury itself

In [9]:
injuries_date_filtered = injuries_original[(injuries_original['Date']>'1996-07-01') & (injuries_original['Date']<'2023-07-01')]

In [10]:
injuries_date_filtered

,Date,Team,Acquired,Relinquished,Notes
2055,1996-09-11,Pacers,NaN,Rik Smits,placed on IR with surgery on feet (date approx...
2056,1996-10-30,Blazers,NaN,Aleksandar Djordjevic,placed on IL with sprained right ankle
2057,1996-10-30,Blazers,NaN,Jermaine O'Neal,placed on IL with sprained left knee / bone co...
2058,1996-10-30,Blazers,NaN,Marcus Brown,placed on IL with strained right groin
2059,1996-10-30,Knicks,NaN,Chris Childs,placed on IR with broken right leg
...,...,...,...,...,...
37662,2023-04-16,Clippers,Marcus Morris,NaN,activated from IL
37663,2023-04-16,Grizzlies,Dillon Brooks,NaN,activated from IL
37664,2023-04-16,Grizzlies,Ja Morant,NaN,activated from IL
37665,2023-04-16,Grizzlies,Jaren Jackson Jr.,NaN,activated from IL


In [11]:
injuries = injuries_date_filtered[~(injuries_date_filtered['Relinquished'].isna())]

In [12]:
injuries

,Date,Team,Acquired,Relinquished,Notes
2055,1996-09-11,Pacers,NaN,Rik Smits,placed on IR with surgery on feet (date approx...
2056,1996-10-30,Blazers,NaN,Aleksandar Djordjevic,placed on IL with sprained right ankle
2057,1996-10-30,Blazers,NaN,Jermaine O'Neal,placed on IL with sprained left knee / bone co...
2058,1996-10-30,Blazers,NaN,Marcus Brown,placed on IL with strained right groin
2059,1996-10-30,Knicks,NaN,Chris Childs,placed on IR with broken right leg
...,...,...,...,...,...
37624,2023-04-09,Grizzlies,NaN,Ja Morant,placed on IL with right hand injury
37625,2023-04-09,Grizzlies,NaN,Jaren Jackson Jr.,placed on IL with sore left elbow
37630,2023-04-09,Kings,NaN,Davion Mitchell,placed on IL with sore left knee
37631,2023-04-09,Knicks,NaN,Trevor Keels,placed on IL with illness


In [13]:
injuries = injuries.drop(columns = 'Acquired')

When combining, have to keep date(convert to season?), team, relinquished, and notes

In [14]:
injuries_by_category = {
    "Head": [
        "placed on IR with concussion",
        "placed on IL with head injury",
        "placed on IR with skull fracture",
        "placed on IR with facial fractures",
        "placed on IR with brain injury",
        "placed on IL with head contusion",
        "placed on IR with fractured skull",
        "placed on IL with eye injury",
        "placed on IR with fractured jaw",
        "placed on IL with nasal fracture"
    ],
    "Chest/Abdomen": [
        "placed on IL with chest contusion",
        "placed on IR with rib fractures",
        "placed on IL with abdominal strain",
        "placed on IR with bruised ribs",
        "placed on IL with torn abdominal muscle",
        "placed on IR with ruptured spleen",
        "placed on IL with intercostal strain",
        "placed on IR with hernia",
        "placed on IL with chest muscle strain",
        "placed on IR with abdominal surgery"
    ],
    "Pelvis/Back": [
        "placed on IR with back surgery",
        "placed on IL with strained lower back",
        "placed on IL with lumbar spine strain and sprain",
        "placed on IR with back injury",
        "placed on IL with back spasms",
        "placed on IR with sore lower back",
        "placed on IR with lumbar spine surgery",
        "placed on IL with lower back injury",
        "placed on IL with lumbar spine strain",
        "placed on IL with lower back spasms"
    ],
    "Arm": [
        "placed on IL with broken left arm",
        "placed on IL with strained right biceps",
        "placed on IR with dislocated left shoulder",
        "placed on IL with fractured right humerus",
        "placed on IR with torn right biceps",
        "placed on IL with strained triceps",
        "placed on IR with broken right arm",
        "placed on IL with sprained right elbow",
        "placed on IR with fractured left forearm",
        "placed on IL with strained left biceps"
    ],
    "Hand/Wrist": [
        "placed on IL with broken right hand",
        "placed on IR with dislocated finger in left hand",
        "placed on IR with fractured right thumb",
        "placed on IL with sprained left wrist",
        "placed on IR with broken finger",
        "placed on IR with torn ligament in right thumb",
        "placed on IR with broken left arm",
        "placed on IR with fractured left pinky finger",
        "placed on IL with fractured bone in right wrist",
        "placed on IR with fractured right index finger"
    ],
    "Leg": [
        "placed on IR with broken right leg",
        "placed on IL with strained right leg",
        "placed on IL with lower right leg stress reaction",
        "placed on IR with sore left leg",
        "placed on IL with stress fracture in left leg",
        "placed on IR with stress fractures in both legs",
        "placed on IR with strained left leg",
        "placed on IR with pulled right hamstring",
        "placed on IL with hamstring injury",
        "placed on IR with chronic pain in lower right leg"
    ],
    "Knee": [
        "placed on IL with sprained left knee / bone contusion in left knee",
        "placed on IL recovering from surgery on knee to repair torn ACL",
        "placed on IR with posterior capsule strain of left knee",
        "placed on IL with patellar tendinitis in left knee",
        "placed on IL with right knee patella tendinitis",
        "placed on IR with left knee surgery",
        "placed on IL with torn ACL",
        "placed on IL with torn cartilage in left knee",
        "placed on IL with knee surgery rehab",
        "placed on IR with torn meniscus in left knee"
    ],
    "Foot/Ankle": [
        "placed on IR with surgery on feet",
        "placed on IL with sprained right ankle",
        "placed on IL with broken bone in right foot",
        "placed on IR with stress fracture in right foot",
        "placed on IR with fractured left foot",
        "placed on IL with plantar fasciitis in left foot",
        "placed on IR with fractured right ankle",
        "placed on IR with stress fracture in left foot",
        "placed on IR with ruptured Achilles tendon",
        "placed on IR with torn plantar fascia in foot"
    ]
}

In [15]:
injuries = injuries.reset_index(drop=True)

In [16]:
injuries

,Date,Team,Relinquished,Notes
0,1996-09-11,Pacers,Rik Smits,placed on IR with surgery on feet (date approx...
1,1996-10-30,Blazers,Aleksandar Djordjevic,placed on IL with sprained right ankle
2,1996-10-30,Blazers,Jermaine O'Neal,placed on IL with sprained left knee / bone co...
3,1996-10-30,Blazers,Marcus Brown,placed on IL with strained right groin
4,1996-10-30,Knicks,Chris Childs,placed on IR with broken right leg
...,...,...,...,...
18887,2023-04-09,Grizzlies,Ja Morant,placed on IL with right hand injury
18888,2023-04-09,Grizzlies,Jaren Jackson Jr.,placed on IL with sore left elbow
18889,2023-04-09,Kings,Davion Mitchell,placed on IL with sore left knee
18890,2023-04-09,Knicks,Trevor Keels,placed on IL with illness


Separate stats by season for categorization(Season starts on October 12th, ends October 11th of next year)

In [17]:
def find_season(date):
  year = int(date[0:4])
  month = int(date[5:7])
  day = int(date[8:10])
  if month<10:
    return year-1
  elif month==10:
    if day<12:
      return year-1
    else:
      return year
  else:
    return year

In [18]:
injuries['Date'] = injuries['Date'].apply(find_season)

In [19]:
injuries

,Date,Team,Relinquished,Notes
0,1995,Pacers,Rik Smits,placed on IR with surgery on feet (date approx...
1,1996,Blazers,Aleksandar Djordjevic,placed on IL with sprained right ankle
2,1996,Blazers,Jermaine O'Neal,placed on IL with sprained left knee / bone co...
3,1996,Blazers,Marcus Brown,placed on IL with strained right groin
4,1996,Knicks,Chris Childs,placed on IR with broken right leg
...,...,...,...,...
18887,2022,Grizzlies,Ja Morant,placed on IL with right hand injury
18888,2022,Grizzlies,Jaren Jackson Jr.,placed on IL with sore left elbow
18889,2022,Kings,Davion Mitchell,placed on IL with sore left knee
18890,2022,Knicks,Trevor Keels,placed on IL with illness


In [20]:
def one_year(season):
  return int(season[0:4])

In [21]:
player_stats['season'] = player_stats['season'].apply(one_year)

In [22]:
player_stats

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,Randy Livingston,HOU,22.0,193.04,94.800728,Louisiana State,USA,1996,2,42,...,3.9,1.5,2.4,0.3,0.042,0.071,0.169,0.487,0.248,1996
1,Gaylon Nickerson,WAS,28.0,190.50,86.182480,Northwestern Oklahoma,USA,1994,2,34,...,3.8,1.3,0.3,8.9,0.030,0.111,0.174,0.497,0.043,1996
2,George Lynch,VAN,26.0,203.20,103.418976,North Carolina,USA,1993,1,12,...,8.3,6.4,1.9,-8.2,0.106,0.185,0.175,0.512,0.125,1996
3,George McCloud,LAL,30.0,203.20,102.058200,Florida State,USA,1989,1,7,...,10.2,2.8,1.7,-2.7,0.027,0.111,0.206,0.527,0.125,1996
4,George Zidek,DEN,23.0,213.36,119.748288,UCLA,USA,1995,1,22,...,2.8,1.7,0.3,-14.1,0.102,0.169,0.195,0.500,0.064,1996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12839,Joel Embiid,PHI,29.0,213.36,127.005760,Kansas,Cameroon,2014,1,3,...,33.1,10.2,4.2,8.8,0.057,0.243,0.370,0.655,0.233,2022
12840,John Butler Jr.,POR,20.0,213.36,86.182480,Florida State,USA,Undrafted,Undrafted,Undrafted,...,2.4,0.9,0.6,-16.1,0.012,0.065,0.102,0.411,0.066,2022
12841,John Collins,ATL,25.0,205.74,102.511792,Wake Forest,USA,2017,1,19,...,13.1,6.5,1.2,-0.2,0.035,0.180,0.168,0.593,0.052,2022
12842,Jericho Sims,NYK,24.0,208.28,113.398000,Texas,USA,2021,2,58,...,3.4,4.7,0.5,-6.7,0.117,0.175,0.074,0.780,0.044,2022


In [23]:
injuries = injuries.rename(columns={"Relinquished": "player_name"})
injuries = injuries.rename(columns={"Date": "season"})

In [24]:
injuries

,season,Team,player_name,Notes
0,1995,Pacers,Rik Smits,placed on IR with surgery on feet (date approx...
1,1996,Blazers,Aleksandar Djordjevic,placed on IL with sprained right ankle
2,1996,Blazers,Jermaine O'Neal,placed on IL with sprained left knee / bone co...
3,1996,Blazers,Marcus Brown,placed on IL with strained right groin
4,1996,Knicks,Chris Childs,placed on IR with broken right leg
...,...,...,...,...
18887,2022,Grizzlies,Ja Morant,placed on IL with right hand injury
18888,2022,Grizzlies,Jaren Jackson Jr.,placed on IL with sore left elbow
18889,2022,Kings,Davion Mitchell,placed on IL with sore left knee
18890,2022,Knicks,Trevor Keels,placed on IL with illness


In [25]:
player_stats['season'] = player_stats['season'].astype(int)
injuries['season'] = injuries['season'].astype(int)
player_stats['player_name'] = player_stats['player_name'].astype(str)
injuries['player_name'] = injuries['player_name'].astype(str)


In [26]:
final = pd.merge( injuries, player_stats, how='outer', left_on=['season', 'player_name'], right_on = ['season', 'player_name'])

In [27]:
final

,season,Team,player_name,Notes,team_abbreviation,age,player_height,player_weight,college,country,...,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct
0,1995,Pacers,Rik Smits,placed on IR with surgery on feet (date approx...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1996,Blazers,Aleksandar Djordjevic,placed on IL with sprained right ankle,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1996,Blazers,Aleksandar Djordjevic,placed on IL with flu,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1996,Blazers,Jermaine O'Neal,placed on IL with sprained left knee / bone co...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1996,Blazers,Marcus Brown,placed on IL with strained right groin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31731,2022,NaN,Joel Embiid,NaN,PHI,29.0,213.36,127.005760,Kansas,Cameroon,...,66.0,33.1,10.2,4.2,8.8,0.057,0.243,0.370,0.655,0.233
31732,2022,NaN,John Butler Jr.,NaN,POR,20.0,213.36,86.182480,Florida State,USA,...,19.0,2.4,0.9,0.6,-16.1,0.012,0.065,0.102,0.411,0.066
31733,2022,NaN,John Collins,NaN,ATL,25.0,205.74,102.511792,Wake Forest,USA,...,71.0,13.1,6.5,1.2,-0.2,0.035,0.180,0.168,0.593,0.052
31734,2022,NaN,Jericho Sims,NaN,NYK,24.0,208.28,113.398000,Texas,USA,...,52.0,3.4,4.7,0.5,-6.7,0.117,0.175,0.074,0.780,0.044


pd.merge not giving ideal values, so we will fix this

In [28]:
injuries = injuries.sort_values(by=['season', 'player_name'])

In [29]:
injuries = injuries.reset_index(drop=True)

In [30]:
injuries

,season,Team,player_name,Notes
0,1995,Pacers,Rik Smits,placed on IR with surgery on feet (date approx...
1,1996,Jazz,Adam Keefe,placed on IR with bruised left hamstring
2,1996,76ers,Adrian Caldwell,placed on IR with bursitis in right foot
3,1996,Hawks,Alan Henderson,placed on IR with acute viral pancreatitis / g...
4,1996,Blazers,Aleksandar Djordjevic,placed on IL with sprained right ankle
...,...,...,...,...
18887,2022,Pelicans,Zion Williamson,placed on IL with bruised right foot
18888,2022,Pelicans,Zion Williamson,placed on IL with NBA health and safety protocols
18889,2022,Pelicans,Zion Williamson,placed on IL with strained right hamstring (ou...
18890,2022,Magic,left knee injury (DTD),placed on IL with left knee injury


Categorize injuries

In [52]:
injuries_by_category = {
    "Head": [
        "placed on IR with concussion",
        "placed on IL with head injury",
        "placed on IR with skull fracture",
        "placed on IR with facial fractures",
        "placed on IR with brain injury",
        "placed on IL with head contusion",
        "placed on IR with fractured skull",
        "placed on IL with eye injury",
        "placed on IR with fractured jaw",
        "placed on IL with nasal fracture"
    ],
    "Chest/Abdomen": [
        "placed on IL with chest contusion",
        "placed on IR with rib fractures",
        "placed on IL with abdominal strain",
        "placed on IR with bruised ribs",
        "placed on IL with torn abdominal muscle",
        "placed on IR with ruptured spleen",
        "placed on IL with intercostal strain",
        "placed on IR with hernia",
        "placed on IL with chest muscle strain",
        "placed on IR with abdominal surgery"
    ],
    "Pelvis/Back": [
        "placed on IR with back surgery",
        "placed on IL with strained lower back",
        "placed on IL with lumbar spine strain and sprain",
        "placed on IR with back injury",
        "placed on IL with back spasms",
        "placed on IR with sore lower back",
        "placed on IR with lumbar spine surgery",
        "placed on IL with lower back injury",
        "placed on IL with lumbar spine strain",
        "placed on IL with lower back spasms"
    ],
    "Arm": [
        "placed on IL with broken left arm",
        "placed on IL with strained right biceps",
        "placed on IR with dislocated left shoulder",
        "placed on IL with fractured right humerus",
        "placed on IR with torn right biceps",
        "placed on IL with strained triceps",
        "placed on IR with broken right arm",
        "placed on IL with sprained right elbow",
        "placed on IR with fractured left forearm",
        "placed on IL with strained left biceps"
    ],
    "Hand/Wrist": [
        "placed on IL with broken right hand",
        "placed on IR with dislocated finger in left hand",
        "placed on IR with fractured right thumb",
        "placed on IL with sprained left wrist",
        "placed on IR with broken finger",
        "placed on IR with torn ligament in right thumb",
        "placed on IR with broken left arm",
        "placed on IR with fractured left pinky finger",
        "placed on IL with fractured bone in right wrist",
        "placed on IR with fractured right index finger"
    ],
    "Leg": [
        "placed on IR with broken right leg",
        "placed on IL with strained right leg",
        "placed on IL with lower right leg stress reaction",
        "placed on IR with sore left leg",
        "placed on IL with stress fracture in left leg",
        "placed on IR with stress fractures in both legs",
        "placed on IR with strained left leg",
        "placed on IR with pulled right hamstring",
        "placed on IL with hamstring injury",
        "placed on IR with chronic pain in lower right leg"
    ],
    "Knee": [
        "placed on IL with sprained left knee / bone contusion in left knee",
        "placed on IL recovering from surgery on knee to repair torn ACL",
        "placed on IR with posterior capsule strain of left knee",
        "placed on IL with patellar tendinitis in left knee",
        "placed on IL with right knee patella tendinitis",
        "placed on IR with left knee surgery",
        "placed on IL with torn ACL",
        "placed on IL with torn cartilage in left knee",
        "placed on IL with knee surgery rehab",
        "placed on IR with torn meniscus in left knee"
    ],
    "Foot/Ankle": [
        "placed on IR with surgery on feet",
        "placed on IL with sprained right ankle",
        "placed on IL with broken bone in right foot",
        "placed on IR with stress fracture in right foot",
        "placed on IR with fractured left foot",
        "placed on IL with plantar fasciitis in left foot",
        "placed on IR with fractured right ankle",
        "placed on IR with stress fracture in left foot",
        "placed on IR with ruptured Achilles tendon",
        "placed on IR with torn plantar fascia in foot"
    ]
}



In [32]:
categorizer_model = spacy.blank('en')
categorizer_model.add_pipe('sentencizer')

  _torch_pytree._register_pytree_node(



In [33]:
with open('listofinjuries.txt', "r") as f:
  text = f.read()
categorizer = categorizer_model(text)

In [55]:
injuries_by_category

{'Head': ['placed on IR with concussion',
  'placed on IL with head injury',
  'placed on IR with skull fracture',
  'placed on IR with facial fractures',
  'placed on IR with brain injury',
  'placed on IL with head contusion',
  'placed on IR with fractured skull',
  'placed on IL with eye injury',
  'placed on IR with fractured jaw',
  'placed on IL with nasal fracture'],
 'Chest/Abdomen': ['placed on IL with chest contusion',
  'placed on IR with rib fractures',
  'placed on IL with abdominal strain',
  'placed on IR with bruised ribs',
  'placed on IL with torn abdominal muscle',
  'placed on IR with ruptured spleen',
  'placed on IL with intercostal strain',
  'placed on IR with hernia',
  'placed on IL with chest muscle strain',
  'placed on IR with abdominal surgery'],
 'Pelvis/Back': ['placed on IR with back surgery',
  'placed on IL with strained lower back',
  'placed on IL with lumbar spine strain and sprain',
  'placed on IR with back injury',
  'placed on IL with back spa

In [61]:
nlp = spacy.load('en_core_web_md')
nlp=spacy.blank('en')
nlp.add_pipe(
    'textcat',
    config ={
        'vocab': injuries_by_category,
        'model': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    }


)

ConfigValidationError: 

Config validation error
textcat -> model	instance of Model expected
textcat -> vocab	extra fields not permitted
{'nlp': <spacy.lang.en.English object at 0x7cb049d67280>, 'name': 'textcat', 'model': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', 'scorer': {'@scorers': 'spacy.textcat_scorer.v2'}, 'threshold': 0.0, 'vocab': {'Head': ['placed on IR with concussion', 'placed on IL with head injury', 'placed on IR with skull fracture', 'placed on IR with facial fractures', 'placed on IR with brain injury', 'placed on IL with head contusion', 'placed on IR with fractured skull', 'placed on IL with eye injury', 'placed on IR with fractured jaw', 'placed on IL with nasal fracture'], 'Chest/Abdomen': ['placed on IL with chest contusion', 'placed on IR with rib fractures', 'placed on IL with abdominal strain', 'placed on IR with bruised ribs', 'placed on IL with torn abdominal muscle', 'placed on IR with ruptured spleen', 'placed on IL with intercostal str

In [38]:
final_data = []
for injury in categorizer.sents:
  injury_listing = nlp(injury.text)
  final_data.append({'listing': injury_listing, 'cats': injury_listing._.cats})

AttributeError: [E046] Can't retrieve unregistered extension attribute 'cats'. Did you forget to call the `set_extension` method?

In [ ]:
injuries_lists = injuries.copy()
listtodrop = []
startindex = 1
for i in range(1, len(injuries)):
  injuries_lists['Notes'][i] = [injuries['Notes'][i]]
  if injuries['player_name'][i] == injuries['player_name'][i-1]:
    listtodrop.append(i)
    injuries_lists['Notes'][startindex].append(injuries['Notes'][i])
  else:
    startindex = i
injuries_lists = injuries_lists.drop(listtodrop, axis='index')

In [ ]:
injuries_lists['Notes'][0] = [injuries['Notes'][0]]

In [ ]:
injuries_lists

In [ ]:
injuries_lists['Notes']

In [ ]:
player_stats = player_stats.sort_values(by = ['season', 'player_name'])

In [ ]:
player_stats = player_stats.reset_index(drop=True)

In [ ]:
player_stats

In [ ]:
player_stats['injuries'] = [[] for i in range(len(player_stats))]

In [ ]:
player_stats

In [ ]:
injuries_lists

In [ ]:
player_stats

In [ ]:
injuries_lists = injuries_lists.reset_index(drop=True)

In [ ]:
injuries_lists

In [ ]:
for i in range(len(injuries_lists)):
  injuries_lists['player_name'][i] = injuries_lists['player_name'][i][1: len(injuries_lists['player_name'][i])]

In [ ]:
player_index= 0
for i in range(1, len(injuries_lists)):
  while player_index<len(player_stats) and player_stats['player_name'][player_index]<=injuries_lists['player_name'][i] and player_stats['season'][player_index]<=injuries_lists['season'][i]:

    if player_stats['player_name'][player_index]==injuries_lists['player_name'][i] and player_stats['season'][player_index]==injuries_lists['season'][i]:
      player_stats['injuries'][player_index] = injuries_lists['Notes'][i]

      player_index+=1
      break
    player_index+=1


In [ ]:
player_stats['injuries'][12843]

In [ ]:
player_stats[player_stats['player_name']=='Zion Williamson']['gp']